In [474]:
import pandas as pd
import os
import altair as alt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
load_dotenv()


True

In [479]:
class SpotifyArtistMetrics:
    def __init__(self, client_id, client_secret):

        self.client_credentials_manager = SpotifyClientCredentials(client_id= client_id, client_secret= client_secret)
        self.sp = spotipy.Spotify(client_credentials_manager=self.client_credentials_manager)

    def get_artist_id(self, artist_name):
        results = self.sp.search(q=artist_name, limit=1, type="artist")
        return results['artists']['items'][0]['id']
    
    def get_artist_albums_and_tracks(self, artist_id):
        albumes = self.sp.artist_albums(artist_id, album_type='album')
        singles = self.sp.artist_albums(artist_id, album_type='single')

        canciones_data = []

        for album in albumes["items"] + singles["items"]:
            album_nombre = album["name"]
            album_tipo = {'album':'Álbum', 'single':'Single'}[album["album_type"]]
            album_año = album["release_date"].split('-')[0]

            tracks = self.sp.album_tracks(album['id'])

            for track in tracks["items"]:
                cancion_nombre = track["name"]
                cancion_artista = ', '.join([t['name'] for t in track["artists"]])
                cancion_duracion = '{:02d}:{:02d}'.format(*divmod(track["duration_ms"]//1000,60))
                cancion_popularidad = self.sp.track(track['id'])["popularity"]     

                canciones_data.append([album_nombre, album_tipo, album_año,
                                        cancion_nombre, cancion_artista, cancion_duracion, cancion_popularidad])

        return pd.DataFrame(canciones_data, columns=["Album", "tipo", "Año", "Cancion",
                                                    "Artistas", "Duracion", "Popularidad"])
    
    def df_chart_track_popularity(self, DataFrame):
        top_popularidad = DataFrame.sort_values("Popularidad", ascending=False).head(30)

        chart = alt.Chart(top_popularidad).mark_bar(color='#1DB954').encode(
            x= alt.X('Cancion', sort= '-y'),
            y= 'Popularidad',
        ).properties(
            width = 700,
            height = 400
        )
        text = chart.mark_text(
        align = 'center',
        baseline= 'top',
        dy = -12,
        fontSize=12
        ).encode(text=alt.Text('Popularidad'))

        return chart + text
    
    def df_chart_top_duration(self,DataFrame):
        DataFrame["Duracion_seg"] = DataFrame["Duracion"].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))
        top_duracion = DataFrame.sort_values("Duracion_seg", ascending=False).head(30)

        chart = alt.Chart(top_duracion).mark_bar(size=15).encode(
            x = alt.X('Cancion', sort='-y'),
            y = 'Duracion_seg'
        ).properties(
            width = 700,
            height = 400
        )
        text = chart.mark_text(
            align = 'center',
            baseline= 'top',
            dy = -12,
            fontSize=9
        ).encode(text=alt.Text('Duracion'))
        return chart + text

In [481]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

metrics = SpotifyArtistMetrics(client_id, client_secret)
id = metrics.get_artist_id('Trivium')
df = metrics.get_artist_albums_and_tracks(id)

In [482]:
chart = metrics.df_chart_track_popularity(df)
chart

alt.LayerChart(...)

In [483]:
chart = metrics.df_chart_top_duration(df)
chart

alt.LayerChart(...)